In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# 2-1: Dense Layers

## Shapes of Dense Layers

In [8]:
from tensorflow.keras.layers import Dense

In [5]:
N, n_feature = 8, 10
X = tf.random.normal(shape=(N, n_feature))

n_neuron = 3
dense = Dense(units=n_neuron, activation='sigmoid')
Y = dense(X)

W, B = dense.get_weights()

print('----- Input/Weights/Bias -----')
print('X: ', X.shape)
print('W: ', W.shape)
print('B: ', B.shape)
print('Y: ', Y.shape)

----- Input/Weights/Bias -----
X:  (8, 10)
W:  (10, 3)
B:  (3,)
Y:  (8, 3)


## Output Calculations

In [9]:
import numpy as np

from tensorflow.math import exp
from tensorflow.linalg import matmul

In [14]:
N, n_feature = 4, 10
X = tf.random.normal(shape=(N, n_feature))

n_neurons = 3
dense = Dense(units=n_neuron, activation='sigmoid')
Y_tf = dense(X)

W, B = dense.get_weights()
print('Y(tensorflow): \n', Y_tf.numpy())

# calculate with matrix multiplication
z = matmul(X, W) + B
Y_man_matmul = 1 / (1+exp(-z))
print('Y(with matrix multiplication): \n', Y_man_matmul.numpy())

# calculate with dot products
Y_man_vec = np.zeros(shape=(N, n_neurons))

for x_idx in range(N):
  x = X[x_idx]

  for nu_idx in range(n_neuron):
    w, b = W[:, nu_idx], B[nu_idx]

    z = tf.reduce_sum(x*w) + b
    a = 1 / (1+np.exp(-z))
    Y_man_vec[x_idx, nu_idx] = a

print('Y(with dot products): \n', Y_man_vec)

Y(tensorflow): 
 [[0.2720297  0.5294572  0.6640466 ]
 [0.7723684  0.5114467  0.5736281 ]
 [0.85403985 0.0182881  0.09786273]
 [0.7787302  0.677236   0.6546691 ]]
Y(with matrix multiplication): 
 [[0.2720297  0.5294572  0.6640466 ]
 [0.7723684  0.5114467  0.5736281 ]
 [0.85403985 0.0182881  0.09786273]
 [0.7787302  0.677236   0.6546691 ]]
Y(with dot products): 
 [[0.27202971 0.52945722 0.66404651]
 [0.77236845 0.51144672 0.57362814]
 [0.8540398  0.0182881  0.09786275]
 [0.77873025 0.67723605 0.65466907]]


# 2-2: Cascaded Dense Layers

## Shapes of Cascaded Dense Layers

In [18]:
N, n_feature = 4, 10
X = tf.random.normal(shape=(N, n_feature))

n_neurons = [3, 5]
dense1 = Dense(units=n_neurons[0], activation='sigmoid')
dense2 = Dense(units=n_neurons[1], activation='sigmoid')

# forward propagation
A1 = dense1(X)
Y = dense2(A1)

W1, B1 = dense1.get_weights()
W2, B2 = dense2.get_weights()

print(f'X: {X.shape}\n')

print(f'W1: {W1.shape}')
print(f'B1: {B1.shape}')
print(f'A1: {A1.shape}\n')

print(f'W1: {W2.shape}')
print(f'B1: {B2.shape}')
print(f'Y: {Y.shape}')

X: (4, 10)

W1: (10, 3)
B1: (3,)
A1: (4, 3)

W1: (3, 5)
B1: (5,)
Y: (4, 5)


## Dense Layers with Python List

In [19]:
N, n_feature = 4, 10
X = tf.random.normal(shape=(N, n_feature))

n_neurons = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

dense_layers = []
for n_neuron in n_neurons:
  dense = Dense(units=n_neuron, activation='relu')
  dense_layers.append(dense)

print('Input: ', X.shape)
for dense_idx, dense in enumerate(dense_layers):
  X = dense(X)
  print('After dense layer', dense_idx+1)
  print(X.shape, '\n')

Input:  (4, 10)
After dense layer 1
(4, 10) 

After dense layer 2
(4, 20) 

After dense layer 3
(4, 30) 

After dense layer 4
(4, 40) 

After dense layer 5
(4, 50) 

After dense layer 6
(4, 60) 

After dense layer 7
(4, 70) 

After dense layer 8
(4, 80) 

After dense layer 9
(4, 90) 

After dense layer 10
(4, 100) 



## Output Calculations

In [24]:
N, n_feature = 4, 10
X = tf.random.normal(shape=(N, n_feature))
X_cp = tf.identity(X)

n_neurons = [3, 4, 5]

dense_layers = []
for n_neuron in n_neurons:
  dense = Dense(units=n_neuron, activation='sigmoid')
  dense_layers.append(dense)

# forward propagation(Tensorflow)
W, B = [], []
for dense_idx, dense in enumerate(dense_layers):
  X = dense(X)
  w, b = dense.get_weights()

  W.append(w)
  B.append(b)
print('Y(Tensorflow):\n', X.numpy())

# forward propagation(manual)
for layer_idx in range(len(n_neurons)):
  w, b = W[layer_idx], B[layer_idx]

  X_cp = matmul(X_cp, w) + b
  X_cp = 1 / (1+exp(-X_cp))
print('Y(Manual):\n', X_cp.numpy())

Y(Tensorflow):
 [[0.60199696 0.68401223 0.4079938  0.4002468  0.5413205 ]
 [0.6228001  0.685824   0.3963667  0.37556857 0.57274616]
 [0.59007865 0.68162346 0.41373533 0.40578207 0.5276527 ]
 [0.59686685 0.679015   0.41420156 0.4105545  0.5354568 ]]
Y(Manual):
 [[0.60199696 0.68401223 0.4079938  0.4002468  0.5413205 ]
 [0.6228001  0.685824   0.3963667  0.37556857 0.57274616]
 [0.59007865 0.68162346 0.41373533 0.40578207 0.5276527 ]
 [0.59686685 0.679015   0.41420156 0.4105545  0.5354568 ]]


# 2-3: Model Implementation

## Model Implementation with Sequential Method

In [25]:
from tensorflow.keras.models import Sequential

In [29]:
model = Sequential()
model.add(Dense(units=10, activation='sigmoid'))
model.add(Dense(units=20, activation='sigmoid'))

## Model Implementation with Model-subclassing

In [30]:
from tensorflow.keras.models import Model

In [33]:
class TestModel(Model):
  def __init__(self):
    super(TestModel, self).__init__()

    self.dense1 = Dense(units=10, activation='sigmoid')
    self.dense2 = Dense(units=20, activation='sigmoid')

model = TestModel()
print(model.dense1)
print(model.dense2)

## Forward Propagation of Models

In [34]:
X = tf.random.normal(shape=(4, 10))

# sequential method
model = Sequential()
model.add(Dense(units=10, activation='sigmoid'))
model.add(Dense(units=20, activation='sigmoid'))

Y = model(X)

# Model-subclassing
class TestModel(Model):
  def __init__(self):
    super(TestModel, self).__init__()

    self.dense1 = Dense(units=10, activation='sigmoid')
    self.dense2 = Dense(units=20, activation='sigmoid')

  def call(self, x):
    x = self.dense1(x)
    x = self.dense2(x)
    return x

model = TestModel()
Y = model(X)

In [ ]:
class TestModel1(Model):
  def __init__(self, n_neurons):
    super(TestModel1, self).__init__()
    self.n_neurons = n_neurons

    self.dense_layers = []
    for n_neuron in self.ne_neurons:
      self.dense_layers.append(Dense(units=n_neuron, activation='sigmoid'))

  def call(self, x):
    for dense in self.dense_layers:
      x = dense(x)
    return x


class TestModel2(Model):
  def __init__(self, n_neurons):
    super(TestModel2, self).__init__()
    self.n_neurons = n_neurons

    model = Sequential()
    model.add(Dense(units=10, activation='sigmoid'))
    model.add(Dense(units=20, activation='sigmoid'))

  def call(self, x):
    x = self.model(x)
    return x

## Layers in Models

In [39]:
X = tf.random.normal(shape=(4, 10))

model = Sequential()
model.add(Dense(units=10, activation='sigmoid'))
model.add(Dense(units=20, activation='sigmoid'))

Y = model(X)

print(type(model.layers))
print(model.layers)

for layer in model.layers:
  w, b = layer.get_weights()
  print(w.shape, b.shape)

<class 'list'>
[<keras.layers.core.dense.Dense object at 0x7f25ebb09310>, <keras.layers.core.dense.Dense object at 0x7f25ebbfb550>]
(10, 10) (10,)
(10, 20) (20,)


In [40]:
dense1 = model.layers[0]
for tmp in dir(dense1):
  print(tmp)

_TF_MODULE_IGNORED_PROPERTIES
__call__
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__getstate__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__setstate__
__sizeof__
__str__
__subclasshook__
__weakref__
_activity_regularizer
_add_trackable
_add_trackable_child
_add_variable_with_custom_getter
_auto_track_sub_layers
_autocast
_autographed_call
_build_input_shape
_call_accepts_kwargs
_call_arg_was_passed
_call_fn_arg_defaults
_call_fn_arg_positions
_call_fn_args
_call_full_argspec
_callable_losses
_cast_single_input
_checkpoint_dependencies
_clear_losses
_compute_dtype
_compute_dtype_object
_dedup_weights
_default_training_arg
_deferred_dependencies
_delete_tracking
_deserialization_dependencies
_deserialize_from_proto
_dtype
_dtype_policy
_dynamic
_eager_losses
_expects_mask_arg
_expects_training_arg
_flatten
_flatten_layers
_flatten_modules
_functional_constru

## Trainable Variables in Models

In [43]:
print(type(model.trainable_variables))
print(len(model.trainable_variables))

<class 'list'>
4


In [44]:
for train_var in model.trainable_variables:
  print(train_var.shape) # weight, bias

(10, 10)
(10,)
(10, 20)
(20,)
